In [1]:
# YOLO v3으로 정지 영상에서 물체 검출하기
import numpy as np
import cv2 as cv
import sys

def construct_yolo_v3():
    f = open('coco_names.txt', 'r')
    class_names = [line.strip() for line in f.readlines()]

    model = cv.dnn.readNet('yolov3.weights', 'yolov3.cfg')
    layer_names = model.getLayerNames()
    out_layers = [layer_names[i-1] for i in model.getUnconnectedOutLayers()]        # getUnconnectedOutLayers()는 출력 레이어의 인덱스를 반환

    return model, out_layers, class_names

def yolo_detect(img, yolo_model, out_layers):
    height, width = img.shape[0], img.shape[1]
    test_img = cv.dnn.blobFromImage(img, 1.0/256, (448, 448), (0, 0, 0), swapRB=True)   # img, scalefactor, size, mean, swapRB

    yolo_model.setInput(test_img)
    output3 = yolo_model.forward(out_layers)

    box, conf, id = [], [], []          # 박스, 신뢰도, 부류 번호
    for output in output3:
        for vec85 in output:
            scores = vec85[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:        # 신뢰도가 50% 이상인 경우만 취함
                center_x, center_y = int(vec85[0]*width), int(vec85[1]*height)
                w, h = int(vec85[2]*width), int(vec85[3]*height)
                x, y = int(center_x-w/2), int(center_y-h/2)
                box.append([x, y, x+w, y+h])
                conf.append(float(confidence))
                id.append(class_id)


    ind = cv.dnn.NMSBoxes(box, conf, 0.5, 0.4)        # box, conf, score_threshold, nms_threshold
    objects = [box[i] + [conf[i]] + [id[i]] for i in range(len(box)) if i in ind]
    return objects

model, out_layers, class_names = construct_yolo_v3()                # YOLO v3 모델 생성
colors = np.random.uniform(0, 255, size=(len(class_names), 3))      # 부류마다 랜덤 색상 지정

img = cv.imread('soccer.jpg')
if img is None:
    sys.exit('Image load failed!')

res = yolo_detect(img, model, out_layers)                           # YOLO 모델로 물체 검출

for i in range(len(res)):                                           # 검출된 물체를 영상에 표시
    x1, y1, x2, y2, confidence, id = res[i]
    text = str(class_names[id]) + '%.3f'%confidence
    cv.rectangle(img, (x1, y1), (x2, y2), colors[id], 2)
    cv.putText(img, text, (x1, y1+30), cv.FONT_HERSHEY_SIMPLEX, 1.5, colors[id], 2)

cv.imshow('Object detection by YOLO v.3', img)

cv.waitKey()
cv.destroyAllWindows()

In [1]:
# YOLO v3으로 비디오에서 물체 검출하기
import numpy as np
import cv2 as cv
import sys

def construct_yolo_v3():
    f = open('coco_names.txt', 'r')
    class_names = [line.strip() for line in f.readlines()]

    model = cv.dnn.readNet('yolov3.weights', 'yolov3.cfg')
    layer_names = model.getLayerNames()
    out_layers = [layer_names[i-1] for i in model.getUnconnectedOutLayers()]        # getUnconnectedOutLayers()는 출력 레이어의 인덱스를 반환

    return model, out_layers, class_names

def yolo_detect(img, yolo_model, out_layers):
    height, width = img.shape[0], img.shape[1]
    test_img = cv.dnn.blobFromImage(img, 1.0/256, (448, 448), (0, 0, 0), swapRB=True)   # img, scalefactor, size, mean, swapRB

    yolo_model.setInput(test_img)
    output3 = yolo_model.forward(out_layers)

    box, conf, id = [], [], []          # 박스, 신뢰도, 부류 번호
    for output in output3:
        for vec85 in output:
            scores = vec85[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:        # 신뢰도가 50% 이상인 경우만 취함
                center_x, center_y = int(vec85[0]*width), int(vec85[1]*height)
                w, h = int(vec85[2]*width), int(vec85[3]*height)
                x, y = int(center_x-w/2), int(center_y-h/2)
                box.append([x, y, x+w, y+h])
                conf.append(float(confidence))
                id.append(class_id)


    ind = cv.dnn.NMSBoxes(box, conf, 0.5, 0.4)        # box, conf, score_threshold, nms_threshold
    objects = [box[i] + [conf[i]] + [id[i]] for i in range(len(box)) if i in ind]
    return objects

model, out_layers, class_names = construct_yolo_v3()                # YOLO v3 모델 생성
colors = np.random.uniform(0, 255, size=(len(class_names), 3))      # 부류마다 랜덤 색상 지정

cap = cv.VideoCapture(0, cv.CAP_DSHOW)
if not cap.isOpened():
    sys.exit('Camera open failed!')

while True:
    ret, frame = cap.read()
    if not ret:
        sys.exit('Exit loop due to camera read error!')
    
    res = yolo_detect(frame, model, out_layers)

    for i in range(len(res)):
        x1, y1, x2, y2, confidence, id = res[i]
        text = str(class_names[id]) + '%.3f'%confidence
        cv.rectangle(frame, (x1, y1), (x2, y2), colors[id], 2)
        cv.putText(frame, text, (x1, y1+30), cv.FONT_HERSHEY_SIMPLEX, 1.5, colors[id], 2)

    cv.imshow('Object detection from video by YOLO v.3', frame)

    key = cv.waitKey(1)
    if key == ord('q'):
        break

cap.release()
cv.destroyAllWindows()

In [3]:
# YOLO v3의 비디오 처리량 측정하기
# YOLO v3으로 비디오에서 물체 검출하기
import numpy as np
import cv2 as cv
import sys

def construct_yolo_v3():
    f = open('coco_names.txt', 'r')
    class_names = [line.strip() for line in f.readlines()]

    model = cv.dnn.readNet('yolov3.weights', 'yolov3.cfg')
    layer_names = model.getLayerNames()
    out_layers = [layer_names[i-1] for i in model.getUnconnectedOutLayers()]        # getUnconnectedOutLayers()는 출력 레이어의 인덱스를 반환

    return model, out_layers, class_names

def yolo_detect(img, yolo_model, out_layers):
    height, width = img.shape[0], img.shape[1]
    test_img = cv.dnn.blobFromImage(img, 1.0/256, (448, 448), (0, 0, 0), swapRB=True)   # img, scalefactor, size, mean, swapRB

    yolo_model.setInput(test_img)
    output3 = yolo_model.forward(out_layers)

    box, conf, id = [], [], []          # 박스, 신뢰도, 부류 번호
    for output in output3:
        for vec85 in output:
            scores = vec85[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:        # 신뢰도가 50% 이상인 경우만 취함
                center_x, center_y = int(vec85[0]*width), int(vec85[1]*height)
                w, h = int(vec85[2]*width), int(vec85[3]*height)
                x, y = int(center_x-w/2), int(center_y-h/2)
                box.append([x, y, x+w, y+h])
                conf.append(float(confidence))
                id.append(class_id)


    ind = cv.dnn.NMSBoxes(box, conf, 0.5, 0.4)        # box, conf, score_threshold, nms_threshold
    objects = [box[i] + [conf[i]] + [id[i]] for i in range(len(box)) if i in ind]
    return objects

model, out_layers, class_names = construct_yolo_v3()                # YOLO v3 모델 생성
colors = np.random.uniform(0, 255, size=(len(class_names), 3))      # 부류마다 랜덤 색상 지정

import time

start = time.time()
n_frame = 0

cap = cv.VideoCapture(0, cv.CAP_DSHOW)
if not cap.isOpened():
    sys.exit('Camera open failed!')

while True:
    ret, frame = cap.read()
    if not ret:
        sys.exit('Exit loop due to camera read error!')
    
    res = yolo_detect(frame, model, out_layers)

    for i in range(len(res)):
        x1, y1, x2, y2, confidence, id = res[i]
        text = str(class_names[id]) + '%.3f'%confidence
        cv.rectangle(frame, (x1, y1), (x2, y2), colors[id], 2)
        cv.putText(frame, text, (x1, y1+30), cv.FONT_HERSHEY_SIMPLEX, 1.5, colors[id], 2)

    cv.imshow('Object detection from video by YOLO v.3', frame)
    n_frame += 1

    key = cv.waitKey(1)
    if key == ord('q'):
        break

end = time.time()
print("The Number of Frames processed: ", n_frame, ', elapsed time: ', end-start, '\nFPS(Frame per second) =', n_frame/(end-start))

cap.release()
cv.destroyAllWindows()

The Number of Frames processed:  19 , elapsed time:  9.185178756713867 
FPS(Frame per second) = 2.068549834820801
